In [75]:
import pandas as pd
import folium
import time
import re
import numpy as np

In [76]:
# read files
domain = pd.read_csv('../../data/raw/domain_preprocessed.csv')

# get valid postcode with properties
postcode_list = list(domain['postcode'].unique())

In [77]:
# ors data by postcode
public = pd.read_csv("../../data/raw/ors-data/public_by_postcode.csv").drop(columns = ['Unnamed: 0'])
public = public.rename(columns={"nearest_duration": "public_duration"})

care = pd.read_csv("../../data/raw/ors-data/care_facility_by_postcode.csv")
care = care.rename(columns={"nearest_duration": "care_duration"})

emergency= pd.read_csv("../../data/raw/ors-data/emergency_by_postcode.csv")
emergency = emergency.rename(columns={"durations": "emergency_duration"})

shopping = pd.read_csv("../../data/raw/ors-data/shopping_centers_by_postcode.csv").drop(columns = ['Unnamed: 0'])
shopping = shopping.rename(columns={"nearest_duration": "shopping_duration"})

station = pd.read_csv("../../data/raw/ors-data/train_station_by_postcode.csv")
station = station.rename(columns={"nearest_duration": "station_duration"})

hospital = pd.read_csv("../../data/raw/ors-data/hospital_by_postcode.csv").drop(columns = ['Unnamed: 0'])
hospital = hospital.rename(columns={"durations": "hospital_duration"})
# external school data by postcode
primary= pd.read_csv('../../data/raw/external-data/postcode_primary_school.csv').rename(columns={"POSTCODE": "postcode"})
high= pd.read_csv('../../data/raw/external-data/postcode_high_school.csv').rename(columns={"POSTCODE": "postcode"})

#external datas by postcode 
'''criminal = pd.read_csv('../../data/raw/external-data/criminal.csv')\
            .rename(columns={"Rate per 100,000 population": "crime_rate",'Postcode':'postcode'})\
            .drop(columns = ['Unnamed: 0'])'''
#income = pd.read_csv('../../data/raw/external-data/income.csv')
property_and_elector = pd.read_csv('../../data/raw/external-data/property_and_elector_by_postcode.csv')



In [78]:
df = pd.merge(public[['postcode','public_duration']],
              care[['postcode','care_duration']],on='postcode', how='outer')
df = pd.merge(df,
              shopping[['postcode','shopping_duration']], on='postcode', how='outer')
df = pd.merge(df,
              station[['postcode','station_duration']],
              on='postcode', how='outer')
df = pd.merge(df,
              hospital[['postcode','hospital_duration']],
              on='postcode', how='outer')

df = pd.merge(df,
              primary,
              on='postcode', how='outer')
df = pd.merge(df,
              high,
              on='postcode', how='outer')

df = pd.merge(df,
              property_and_elector,
              on='postcode', how='outer')
df = pd.merge(df,
              emergency[['postcode','emergency_duration']],on='postcode', how='outer')


'''df = pd.merge(df,
              criminal,
              on='postcode', how='outer')
'''

"df = pd.merge(df,\n              criminal,\n              on='postcode', how='outer')\n"

In [79]:
# remove external data not in domain postcode list
df_filtered = df.loc[df['postcode'].isin(postcode_list)]
df_filtered.replace("none", np.nan, inplace=True)

In [83]:
df_filtered

,postcode,public_duration,care_duration,shopping_duration,station_duration,hospital_duration,1-10_x,11-50_x,51-100_x,101-150_x,150+_x,1-10_y,11-50_y,51-100_y,101-150_y,150+_y,property_count,elector_count,emergency_duration
0,3131,258.47,63.46,280.66,185.57,247.6,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,9685.0,14758.0,189.6
1,3939,899.14,326.22,NaN,NaN,730.7,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,2.0,8923.0,12193.0,617.7
2,3429,408.4,263.03,376.96,325.36,582.8,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,4.0,15253.0,27933.0,372.8
3,3144,111.39,41.85,288.87,144.09,82.7,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,5276.0,8317.0,103.6
4,3338,561.47,130.73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,2.0,11175.0,18240.0,386.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,3003,NaN,294.45,NaN,347.54,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,1.0,3068.0,3265.0,370.0
176,3008,87.5,96.34,207.21,230.16,205.6,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,6312.0,5355.0,212.8
177,3207,342.7,130.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9251.0,12477.0,151.6
178,3032,241.01,181.30,372.4,365.14,194.2,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,1.0,13659.0,20396.0,NaN


In [81]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 0 to 179
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   postcode            180 non-null    int64  
 1   public_duration     130 non-null    object 
 2   care_duration       180 non-null    float64
 3   shopping_duration   88 non-null     object 
 4   station_duration    107 non-null    object 
 5   hospital_duration   95 non-null     float64
 6   1-10_x              72 non-null     float64
 7   11-50_x             72 non-null     float64
 8   51-100_x            72 non-null     float64
 9   101-150_x           72 non-null     float64
 10  150+_x              72 non-null     float64
 11  1-10_y              156 non-null    float64
 12  11-50_y             156 non-null    float64
 13  51-100_y            156 non-null    float64
 14  101-150_y           156 non-null    float64
 15  150+_y              156 non-null    float64
 16  property

In [82]:
df_filtered.to_csv("../../data/curated/joined_external.csv")